In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

# 이미지 데이터셋, 전처리, 전이학습 모듈
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from torchvision.models import resnet18, ResNet18_Weights

from torchinfo import summary
from torchmetrics.functional.classification import multiclass_accuracy

In [8]:
# 데이터 준비
img_dir = "../data/img/test"
preprocessing = transforms.Compose(transforms=[
    transforms.Resize(256, interpolation=transforms.InterpolationMode.BILINEAR), 
    transforms.CenterCrop(224), 
    transforms.ToTensor(), 
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

# 이미지 데이터셋 생성
imgDS = ImageFolder(root=img_dir, transform=preprocessing)
print(imgDS.classes, imgDS.targets, imgDS.imgs, sep="\n")

['cats', 'dogs']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[('../data/img/test\\cats\\cat_1.jpg', 0), ('../data/img/test\\cats\\cat_106.jpg', 0), ('../data/img/test\\cats\\cat_109.jpg', 0), ('../data/img/test\\cats\\cat_113.jpg', 0), ('../data/img/test\\cats\\cat_114.jpg', 0), ('../data/img/test\\cats\\cat_116.jpg', 0), ('../data/img/test\\cats\\cat_118.jpg', 0), ('../data/img/test\\cats\\cat_119.jpg', 0), ('../data/img/test\\cats\\cat_124.jpg', 0), ('../data/img/test\\cats\\cat_140.jpg', 0), ('../data/img/test\\cats\\cat_147.jpg', 0), ('../data/img/test\\cats\\cat_156.jpg', 0), ('../data/img/test\\cats\\cat_158.j

In [9]:
imgDL = DataLoader(imgDS, batch_size=3, shuffle=True, drop_last=True)
for (img, label) in imgDL:
    print(img.shape, label)

torch.Size([3, 3, 224, 224]) tensor([0, 0, 0])
torch.Size([3, 3, 224, 224]) tensor([0, 1, 0])
torch.Size([3, 3, 224, 224]) tensor([0, 1, 0])
torch.Size([3, 3, 224, 224]) tensor([0, 1, 1])
torch.Size([3, 3, 224, 224]) tensor([1, 1, 0])
torch.Size([3, 3, 224, 224]) tensor([1, 1, 0])
torch.Size([3, 3, 224, 224]) tensor([1, 1, 0])
torch.Size([3, 3, 224, 224]) tensor([0, 0, 0])
torch.Size([3, 3, 224, 224]) tensor([1, 0, 0])
torch.Size([3, 3, 224, 224]) tensor([1, 0, 1])
torch.Size([3, 3, 224, 224]) tensor([0, 0, 1])
torch.Size([3, 3, 224, 224]) tensor([1, 0, 0])
torch.Size([3, 3, 224, 224]) tensor([0, 0, 0])
torch.Size([3, 3, 224, 224]) tensor([1, 1, 1])
torch.Size([3, 3, 224, 224]) tensor([0, 1, 1])
torch.Size([3, 3, 224, 224]) tensor([0, 1, 0])
torch.Size([3, 3, 224, 224]) tensor([0, 0, 1])
torch.Size([3, 3, 224, 224]) tensor([0, 0, 1])
torch.Size([3, 3, 224, 224]) tensor([1, 1, 0])
torch.Size([3, 3, 224, 224]) tensor([1, 0, 1])
torch.Size([3, 3, 224, 224]) tensor([1, 0, 1])
torch.Size([3

In [11]:
### 모델 설계 및 설정

# 사전 학습된 모델 인스턴스 생성
res_model = resnet18(weights = ResNet18_Weights.DEFAULT)

# 전결합층 변경
# in_features : FeatureMap에서 받은 피쳐 수
# out_features : out_features : 출력/분류 클래스 수
res_model.fc = nn.Linear(in_features=512, out_features=2)

In [12]:
summary(res_model, input_size=(3, 3, 224, 224))

Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   [3, 2]                    --
├─Conv2d: 1-1                            [3, 64, 112, 112]         9,408
├─BatchNorm2d: 1-2                       [3, 64, 112, 112]         128
├─ReLU: 1-3                              [3, 64, 112, 112]         --
├─MaxPool2d: 1-4                         [3, 64, 56, 56]           --
├─Sequential: 1-5                        [3, 64, 56, 56]           --
│    └─BasicBlock: 2-1                   [3, 64, 56, 56]           --
│    │    └─Conv2d: 3-1                  [3, 64, 56, 56]           36,864
│    │    └─BatchNorm2d: 3-2             [3, 64, 56, 56]           128
│    │    └─ReLU: 3-3                    [3, 64, 56, 56]           --
│    │    └─Conv2d: 3-4                  [3, 64, 56, 56]           36,864
│    │    └─BatchNorm2d: 3-5             [3, 64, 56, 56]           128
│    │    └─ReLU: 3-6                    [3, 64, 56, 56]           --
│

In [14]:
# Resnet18 Feature Module 파라미터 requires_grad = True => False로 비활성화
for name, param in res_model.named_parameters():
    print(name, param.requires_grad, end="   ====>   ")
    param.requires_grad = False
    print(param.requires_grad)
    
# Resnet18 Full Connected Module 파라미터 requires_grad = False -> True로 활성화
for name, param in res_model.named_parameters():
    print(name, param.requires_grad, end="   ====>   ")
    param.requires_grad = True
    print(param.requires_grad)

conv1.weight True   ====>   False
bn1.weight True   ====>   False
bn1.bias True   ====>   False
layer1.0.conv1.weight True   ====>   False
layer1.0.bn1.weight True   ====>   False
layer1.0.bn1.bias True   ====>   False
layer1.0.conv2.weight True   ====>   False
layer1.0.bn2.weight True   ====>   False
layer1.0.bn2.bias True   ====>   False
layer1.1.conv1.weight True   ====>   False
layer1.1.bn1.weight True   ====>   False
layer1.1.bn1.bias True   ====>   False
layer1.1.conv2.weight True   ====>   False
layer1.1.bn2.weight True   ====>   False
layer1.1.bn2.bias True   ====>   False
layer2.0.conv1.weight True   ====>   False
layer2.0.bn1.weight True   ====>   False
layer2.0.bn1.bias True   ====>   False
layer2.0.conv2.weight True   ====>   False
layer2.0.bn2.weight True   ====>   False
layer2.0.bn2.bias True   ====>   False
layer2.0.downsample.0.weight True   ====>   False
layer2.0.downsample.1.weight True   ====>   False
layer2.0.downsample.1.bias True   ====>   False
layer2.1.conv1.wei

In [15]:
# 학습 준비
optimizer = optim.Adam(res_model.fc.parameters()) # fc(전결합층)만 줘야함!!
loss_fn = nn.CrossEntropyLoss() # 손실함수 정의
EPOCHS = 3

In [ ]:
# 초기화 시킬 가중치 받아서 클래스로 만들어도 됨 
# 가중치를 None주고 내 데이터로 학습 시켜도 됨(추가 학습)